In [5]:
from pandas.core.frame import DataFrame

In [6]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import (GradientBoostingClassifier, GradientBoostingRegressor,RandomForestClassifier, RandomForestRegressor)
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import learning_curve
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, learning_curve
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
# from xgboost import XGBClassifier
from sklearn.metrics import precision_score

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [9]:
import pandas as pd
excelFile = r'Geri-DL-data/data-dictionary.xlsx'
MeasuresData = pd.DataFrame(pd.read_excel(excelFile,sheet_name='Measures'))

In [10]:
print(MeasuresData.columns.size) # how many colums
print(MeasuresData.iloc[:,0].size) # how many rows

191
13422


In [11]:
MeasuresData=MeasuresData[MeasuresData['Entry-discharge-type']=='Not entry/discharge record'].reset_index(drop=True)
print(MeasuresData.columns.size) # how many colums
print(MeasuresData.iloc[:,0].size) # how many rows

191
7584


In [12]:
import time  
import datetime
def caltimes (date1, date2):
    date1=time.strptime(date1,"%m/%d/%Y")
    date2=time.strptime(date2,"%m/%d/%Y") 
    date1=datetime.datetime(date1[0],date1[1],date1[2])  
    date2=datetime.datetime(date2[0],date2[1],date2[2])
    return (date1-date2).days

In [13]:
a1=caltimes('8/19/2013','7/19/2013')
a1

31

In [14]:
rows=MeasuresData.iloc[:,0]
eps=MeasuresData["episode-start-date"]
epe=MeasuresData["episode-end-date"]
epiday=[]
for i in range(MeasuresData.iloc[:,0].size):
            date=caltimes(epe[i],eps[i])
            epiday.append(date)
                               

In [15]:
epidays=DataFrame(epiday)

In [16]:
epidays.columns= ['episodeTime']

In [18]:
# epidays

In [19]:
stayDay=[]
sts=MeasuresData["stay-start-date"]
ste=MeasuresData["stay-end-date"]
for i in range(MeasuresData.iloc[:,0].size):
    day=caltimes(ste[i],sts[i])
    stayDay.append(day)

In [20]:
stayDays=DataFrame(stayDay)

In [21]:
stayDays.columns=['stayDays']

In [24]:
cmslongstay=MeasuresData['cms-long-stay'].map({'No':0,'Yes':1})

In [26]:
shortTLong=MeasuresData['mds-stay-trans-short-to-long'].map({'No':0,'Yes':1})

In [27]:
dataset=pd.concat([pd.DataFrame(MeasuresData['patient-id']),epidays,stayDays,pd.DataFrame(MeasuresData['current-stay-days']),pd.DataFrame(MeasuresData['cumulative-days-in-facility']),pd.DataFrame(cmslongstay),pd.DataFrame(shortTLong)], axis=1)

In [30]:
gender=MeasuresData['gender'].map({'Male':0,'Female':1})

In [31]:
dataset=pd.concat([dataset,pd.DataFrame(MeasuresData['age']),gender], axis=1)

In [33]:
MeasuresData['race'].value_counts()

White    6130
Black    1296
Other     158
Name: race, dtype: int64

In [34]:
# from keras.utils import to_categorical
# from sklearn import preprocessing

In [35]:
onehot_race = pd.get_dummies(MeasuresData['race'])
dataset=dataset.join(onehot_race)

In [37]:
MeasuresData['facility'].value_counts()

Cranberry Place     1998
Senaca Place        1987
Heritage Place      1640
Canterbury Place    1041
Sugar Creek          918
Name: facility, dtype: int64

In [38]:
onehot_facility = pd.get_dummies(MeasuresData['facility'])
dataset=dataset.join(onehot_facility)

In [39]:
MeasuresData['mds-antianxiety-medication'].value_counts()

No     6398
Yes    1186
Name: mds-antianxiety-medication, dtype: int64

In [40]:
antianxiety=MeasuresData['mds-antianxiety-medication'].map({'No':0,'Yes':1})

In [41]:
dataset=pd.concat([dataset,pd.DataFrame(antianxiety)], axis=1)

In [42]:
MeasuresData['mds-antidepressant-medication'].value_counts()

Yes    4092
No     3492
Name: mds-antidepressant-medication, dtype: int64

In [43]:
antidepressant=MeasuresData['mds-antidepressant-medication'].map({'No':0,'Yes':1})

In [44]:
dataset=pd.concat([dataset,pd.DataFrame(antidepressant)], axis=1)

In [45]:
MeasuresData['mds-antipsychotic-medication'].value_counts()

No     6259
Yes    1325
Name: mds-antipsychotic-medication, dtype: int64

In [46]:
antipsychotic=MeasuresData['mds-antipsychotic-medication'].map({'No':0,'Yes':1})
dataset=pd.concat([dataset,pd.DataFrame(antipsychotic)], axis=1)

In [47]:
MeasuresData['mds-antibiotic-medication'].value_counts()

No     6095
Yes    1489
Name: mds-antibiotic-medication, dtype: int64

In [48]:
antibiotic=MeasuresData['mds-antibiotic-medication'].map({'No':0,'Yes':1})
dataset=pd.concat([dataset,pd.DataFrame(antibiotic)], axis=1)

In [49]:
MeasuresData['mds-anticoagulant-medication'].value_counts()

No     5478
Yes    2106
Name: mds-anticoagulant-medication, dtype: int64

In [50]:
anticoagulant=MeasuresData['mds-anticoagulant-medication'].map({'No':0,'Yes':1})
dataset=pd.concat([dataset,pd.DataFrame(anticoagulant)], axis=1)

In [51]:
MeasuresData['mds-diuretic-medication'].value_counts()

No     5159
Yes    2425
Name: mds-diuretic-medication, dtype: int64

In [53]:
diuretic=MeasuresData['mds-diuretic-medication'].map({'No':0,'Yes':1})
dataset=pd.concat([dataset,pd.DataFrame(diuretic)], axis=1)

In [54]:
MeasuresData['mds-hypnotic-medication'].value_counts()

No     7285
Yes     299
Name: mds-hypnotic-medication, dtype: int64

In [55]:
hypnotic=MeasuresData['mds-hypnotic-medication'].map({'No':0,'Yes':1})
dataset=pd.concat([dataset,pd.DataFrame(hypnotic)], axis=1)

In [56]:
MeasuresData['mds-behavioral-symptoms'].value_counts()

No     7506
Yes      78
Name: mds-behavioral-symptoms, dtype: int64

In [57]:
behavioral=MeasuresData['mds-behavioral-symptoms'].map({'No':0,'Yes':1})
dataset=pd.concat([dataset,pd.DataFrame(behavioral)], axis=1)

In [58]:
MeasuresData['mds-behavioral-symptoms-to-others'].value_counts()

No     6621
Yes     963
Name: mds-behavioral-symptoms-to-others, dtype: int64

In [59]:
behavioral_o=MeasuresData['mds-behavioral-symptoms-to-others'].map({'No':0,'Yes':1})
dataset=pd.concat([dataset,pd.DataFrame(behavioral_o)], axis=1)

In [60]:
MeasuresData['mds-bims-summary-ranking'].value_counts()

Intact or Moderately Intact    6685
Moderate Impairment             899
Name: mds-bims-summary-ranking, dtype: int64

In [61]:
onehot_bims = pd.get_dummies(MeasuresData['mds-bims-summary-ranking'])
dataset=dataset.join(onehot_bims)

In [62]:
MeasuresData['mds-dehydrated'].value_counts()

No     7541
Yes      43
Name: mds-dehydrated, dtype: int64

In [63]:
dehydrated=MeasuresData['mds-dehydrated'].map({'No':0,'Yes':1})
dataset=pd.concat([dataset,pd.DataFrame(dehydrated)], axis=1)

In [64]:
MeasuresData['mds-depression'].value_counts()

No     7315
Yes     269
Name: mds-depression, dtype: int64

In [65]:
depression=MeasuresData['mds-depression'].map({'No':0,'Yes':1})
dataset=pd.concat([dataset,pd.DataFrame(depression)], axis=1)

In [66]:
MeasuresData['mds-fever'].value_counts()

No     7445
Yes     139
Name: mds-fever, dtype: int64

In [67]:
fever=MeasuresData['mds-fever'].map({'No':0,'Yes':1})
dataset=pd.concat([dataset,pd.DataFrame(fever)], axis=1)

In [68]:
MeasuresData['mds-impaired-mobility'].value_counts()

Yes    6805
No      779
Name: mds-impaired-mobility, dtype: int64

In [69]:
impairedM=MeasuresData['mds-impaired-mobility'].map({'No':0,'Yes':1})
dataset=pd.concat([dataset,pd.DataFrame(impairedM)], axis=1)

In [70]:
MeasuresData['mds-impaired-transfer'].value_counts()

Yes    6506
No     1078
Name: mds-impaired-transfer, dtype: int64

In [71]:
impairedT=MeasuresData['mds-impaired-transfer'].map({'No':0,'Yes':1})
dataset=pd.concat([dataset,pd.DataFrame(impairedT)], axis=1)

In [72]:
MeasuresData['mds-internal-bleeding'].value_counts()

No     7475
Yes     109
Name: mds-internal-bleeding, dtype: int64

In [73]:
bleed=MeasuresData['mds-internal-bleeding'].map({'No':0,'Yes':1})
dataset=pd.concat([dataset,pd.DataFrame(bleed)], axis=1)

In [74]:
MeasuresData['mds-pain-non-verbal'].value_counts()

None/Mild          7098
Moderate/Severe     486
Name: mds-pain-non-verbal, dtype: int64

In [75]:
pain=MeasuresData['mds-pain-non-verbal'].map({'None/Mild':0,'Moderate/Severe':1})
dataset=pd.concat([dataset,pd.DataFrame(pain)], axis=1)

In [76]:
# dataset

In [77]:
MeasuresData['mds-malnutrition'].value_counts()

No     7333
Yes     251
Name: mds-malnutrition, dtype: int64

In [78]:
malnutrition=MeasuresData['mds-malnutrition'].map({'No':0,'Yes':1})
dataset=pd.concat([dataset,pd.DataFrame(malnutrition)], axis=1)

In [79]:
MeasuresData['mds-no-problem-conditions'].value_counts()

Yes    7035
No      549
Name: mds-no-problem-conditions, dtype: int64

In [80]:
npconditions=MeasuresData['mds-no-problem-conditions'].map({'No':0,'Yes':1})
dataset=pd.concat([dataset,pd.DataFrame(npconditions)], axis=1)

In [81]:
MeasuresData['mds-vomiting'].value_counts()

No     7383
Yes     201
Name: mds-vomiting, dtype: int64

In [82]:
vomiting=MeasuresData['mds-vomiting'].map({'No':0,'Yes':1})
dataset=pd.concat([dataset,pd.DataFrame(vomiting)], axis=1)

In [83]:
MeasuresData['ws-antibiotic-anticoag-coexposure'].value_counts()

No     7318
Yes     266
Name: ws-antibiotic-anticoag-coexposure, dtype: int64

In [84]:
antibiotic=MeasuresData['ws-antibiotic-anticoag-coexposure'].map({'No':0,'Yes':1})
dataset=pd.concat([dataset,pd.DataFrame(antibiotic)], axis=1)

In [85]:
MeasuresData['ws-psychotropic-with-not-ordered-weight-loss'].value_counts()

No     7098
Yes     486
Name: ws-psychotropic-with-not-ordered-weight-loss, dtype: int64

In [86]:
psychotropic=MeasuresData['ws-psychotropic-with-not-ordered-weight-loss'].map({'No':0,'Yes':1})
dataset=pd.concat([dataset,pd.DataFrame(psychotropic)], axis=1)

In [87]:
MeasuresData['ws-diuretic-adl'].value_counts()

No     5382
Yes    2202
Name: ws-diuretic-adl, dtype: int64

In [88]:
diuretic=MeasuresData['ws-diuretic-adl'].map({'No':0,'Yes':1})
dataset=pd.concat([dataset,pd.DataFrame(diuretic)], axis=1)

In [89]:
MeasuresData['ws-meclizine-psych-coexposure'].value_counts()

No                        7553
meclAndPsychOlderStart      15
meclAndPsychStartSameR      10
psychPrecedesMeclR           3
meclPrecedesPsychR           3
Name: ws-meclizine-psych-coexposure, dtype: int64

In [90]:
onehot_psych = pd.get_dummies(MeasuresData['ws-meclizine-psych-coexposure'])
dataset=dataset.join(onehot_psych)

In [91]:
MeasuresData['ws-tramadol-antidepressant-coexposure'].value_counts()

No     7512
Yes      72
Name: ws-tramadol-antidepressant-coexposure, dtype: int64

In [92]:
tramadol=MeasuresData['ws-tramadol-antidepressant-coexposure'].map({'No':0,'Yes':1})
dataset=pd.concat([dataset,pd.DataFrame(tramadol)], axis=1)

In [93]:
MeasuresData['cns-drug-exposure'].value_counts()

0    2471
1    2430
2    1592
3     735
4     266
5      63
6      23
7       3
8       1
Name: cns-drug-exposure, dtype: int64

In [94]:
dataset=pd.concat([dataset,pd.DataFrame(MeasuresData['cns-drug-exposure'])], axis=1)

In [95]:
MeasuresData['psychotropic-exposure'].value_counts()

0    2881
1    2501
2    1426
3     557
4     168
5      31
6      18
7       1
8       1
Name: psychotropic-exposure, dtype: int64

In [96]:
dataset=pd.concat([dataset,pd.DataFrame(MeasuresData['psychotropic-exposure'])], axis=1)

In [97]:
MeasuresData['pk-interaction'].value_counts()

0    7400
1     176
2       8
Name: pk-interaction, dtype: int64

In [98]:
dataset=pd.concat([dataset,pd.DataFrame(MeasuresData['pk-interaction'])], axis=1)

In [99]:
MeasuresData['outcome-Mds-fall-since-prior-assessment'].value_counts()

None    6192
Yes     1392
Name: outcome-Mds-fall-since-prior-assessment, dtype: int64

In [100]:
assessment=MeasuresData['outcome-Mds-fall-since-prior-assessment'].map({'None':0,'Yes':1})

In [101]:
MeasuresData['outcome-riskmaster-fall-incident'].value_counts()

No     7567
Yes      17
Name: outcome-riskmaster-fall-incident, dtype: int64

In [102]:
riskmaster=MeasuresData['outcome-riskmaster-fall-incident'].map({'No':0,'Yes':1})
outcomes=pd.concat([pd.DataFrame(assessment),pd.DataFrame(riskmaster)], axis=1)

In [103]:
MeasuresData['PPS-assessment'].value_counts()  ### which encode?

None required                                                                                                       5601
5-day scheduled assessment                                                                                          1714
Unscheduled assessment used for PPS (OMRA, significant or clinical change, or significant correction assessment)     109
14-day scheduled assessment                                                                                           54
30-day scheduled assessment                                                                                           41
Readmission/return assessment                                                                                         34
90-day scheduled assessment                                                                                           18
60-day scheduled assessment                                                                                           13
Name: PPS-assessment, dtype: int

In [104]:
onehot_PPS = pd.get_dummies(MeasuresData['PPS-assessment'])
dataset=dataset.join(onehot_PPS)

In [106]:
MeasuresData['Federal-assessment'].value_counts()

Admission assessment (required by day 14)    3092
Quarterly review assessment                  3018
Annual assessment                             616
None required                                 578
Significant change in status assessment       280
Name: Federal-assessment, dtype: int64

In [107]:
onehot_Federal=MeasuresData['Federal-assessment'].map({'None required':0,'Admission assessment (required by day 14)':1,'Quarterly review assessment':2,'Annual assessment':3,'Significant change in status assessment':4})
dataset=dataset.join(onehot_Federal)

In [108]:
# MeasuresData['Entry-discharge-type'].value_counts()

In [109]:
# onehot_type = pd.get_dummies(MeasuresData['Entry-discharge-type'])
# dataset=dataset.join(onehot_type)

In [110]:
MeasuresData['mds-impaired-walk-in-room'].value_counts()

Yes    5695
No     1889
Name: mds-impaired-walk-in-room, dtype: int64

In [111]:
room=MeasuresData['mds-impaired-walk-in-room'].map({'No':0,'Yes':1})
dataset=pd.concat([dataset,pd.DataFrame(room)], axis=1)

In [112]:
MeasuresData['mds-impaired-walk-in-corridor'].value_counts()

Yes    5945
No     1639
Name: mds-impaired-walk-in-corridor, dtype: int64

In [113]:
corridor=MeasuresData['mds-impaired-walk-in-corridor'].map({'No':0,'Yes':1})
dataset=pd.concat([dataset,pd.DataFrame(corridor)], axis=1)

In [115]:
MeasuresData['mds-impaired-locomot-unit'].value_counts()

Yes    5744
No     1840
Name: mds-impaired-locomot-unit, dtype: int64

In [116]:
locomot=MeasuresData['mds-impaired-locomot-unit'].map({'No':0,'Yes':1})
dataset=pd.concat([dataset,pd.DataFrame(locomot)], axis=1)

In [117]:
MeasuresData['mds-impaired-locomot-other'].value_counts()

Yes    6107
No     1477
Name: mds-impaired-locomot-other, dtype: int64

In [118]:
other=MeasuresData['mds-impaired-locomot-other'].map({'No':0,'Yes':1})
dataset=pd.concat([dataset,pd.DataFrame(other)], axis=1)

In [119]:
MeasuresData['outcome-mds-fall-no-injury'].value_counts()

None           6500
One             678
Two or more     406
Name: outcome-mds-fall-no-injury, dtype: int64

In [120]:
noInjury=MeasuresData['outcome-mds-fall-no-injury'].map({'None':0,'One':1,'Two or more':1})
# dataset=pd.concat([dataset,pd.DataFrame(noInjury)], axis=1)
outcomes=pd.concat([outcomes,pd.DataFrame(noInjury)], axis=1)

In [122]:
minorInjury=MeasuresData['outcome-mds-fall-minor-injury'].map({'None':0,'One':1,'Two or more':1}) ## change to none and one or more
# dataset=pd.concat([dataset,pd.DataFrame(minorInjury)], axis=1)
outcomes=pd.concat([outcomes,pd.DataFrame(minorInjury)], axis=1)

In [123]:
MeasuresData['outcome-mds-fall-major-injury'].value_counts()

None    7531
One       53
Name: outcome-mds-fall-major-injury, dtype: int64

In [124]:
majorInjury=MeasuresData['outcome-mds-fall-major-injury'].map({'None':0,'One':1})
outcomes=pd.concat([outcomes,pd.DataFrame(majorInjury)], axis=1)

In [125]:
## save encoding features and outcomes into excel files

dataset.to_excel(r'Geri-DL-data/EncodingFeature_NoMissing.xlsx')
outcomes.to_excel(r'Geri-DL-data/EncodingOutcomes_NoMissing.xlsx')